# Tarea 2 - Procesamiento distribuido y redes neuronales profundas
**Integrantes:** 

Camila Goméz Nazal -
Ignacio Zurita Tapia

In [109]:
import torch
import numpy as np
from torchvision.datasets import DatasetFolder
from torchvision.transforms import Compose, RandomHorizontalFlip, Resize, RandomRotation, Lambda, ToTensor, CenterCrop, Normalize
from torch.utils.data import DataLoader, Sampler, RandomSampler
from torch.utils.data.sampler import SubsetRandomSampler
from IPython.display import HTML
from PIL import Image
from torchvision import get_image_backend
import cv2
seed = 81818 
torch.random.manual_seed(seed)

## Carga y transformación de datos


#### 1. Dataset Folder

In [86]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
            
def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def loader(path):
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

In [87]:
scale = Lambda(lambda x: torch.div(x, torch.max(x)))
brightness = Lambda(lambda x: torch.mul(x, torch.FloatTensor(x.shape[0], x.shape[1], x.shape[2]).uniform_(1.2, 1.5)))

In [97]:
root = 'ChestXRay2017/chest_xray'
extensions = ('.jpeg')
transform = Compose([Resize([224,224]),
                     RandomHorizontalFlip(0.5),
                     RandomRotation([-20,20]),
                     ToTensor(),
                     brightness,
                     scale
                    ])

train = DatasetFolder(root + '/train', loader, extensions, transform)
test = DatasetFolder(root + '/test', loader, extensions, transform)

#### 2. Cantidad de muestras en cada clase en train y test:



In [98]:
HTML(filename='distribucion_datos.html')

,Normal,NeumonÃ­a
Train,1349,3884
Test,234,390


#### 3.

In [111]:
etiquetas_prueba = list(zip(*test.samples))[1]
l1=len(test.samples)
ind_test = np.random.choice(l1, l1, replace=False )
ind_test.shape

(624,)

In [112]:
l =len(train.samples)
np.random.seed(0)
indices = np.random.choice(l, l, replace=False)
indices.shape

(5232,)

In [113]:
ind_train = indices[:int(0.8*l)]
ind_val   = indices[int(0.8*l):]
print(ind_train.shape)
print(ind_val.shape)

(4185,)
(1047,)


In [102]:
val_samples = [train.samples[i] for i in ind_val]
etiquetas_val  = list(zip(*val_samples))[1]
print(etiquetas_val.count(0))
print(etiquetas_val.count(1))

263
784


In [103]:
class ReplicarMuestreoDePrueba(Sampler):
    def __init__(self, etiquetas_prueba, indices_val, etiquetas_val):
        self.etiquetas_prueba = etiquetas_prueba
        self.indices_val      = indices_val
        self.etiquetas_val    = etiquetas_val
    def __iter__(self):
        dist_test   = self.etiquetas_prueba.count(0)/self.etiquetas_prueba.count(1)
        dist_val    = int(self.etiquetas_val.count(1)*dist_test)
        nro_etiq_nuevas = dist_val - self.etiquetas_val.count(0) 
        c=0
        for i in range(len(etiquetas_val)):
            if etiquetas_val[i]==0:
                self.indices_val.append(self.indices_val[i])
                c+=1
            if c==nro_etiq_nuevas:
                break
        return iter(self.indices_val)

c = ReplicarMuestreoDePrueba(etiquetas_prueba,list(ind_val),etiquetas_val)
it = c.__iter__()

#### 4.

In [104]:
train_sampler = SubsetRandomSampler(ind_train)
valid_sampler = SubsetRandomSampler(list(it))
test_sampler = RandomSampler(ind_test)


train_loader = torch.utils.data.DataLoader(train, batch_size=16, 
                                           sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train, batch_size=16,
                                                sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test, batch_size=16, 
                                           sampler=test_sampler)

print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

262
79
39


In [117]:
train_loader.dataset

Dataset DatasetFolder
    Number of datapoints: 5232
    Root location: ChestXRay2017/chest_xray/train
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-20, 20], resample=False, expand=False)
               ToTensor()
               Lambda()
               Lambda()
           )

In [106]:
import matplotlib.pyplot as plt
plt.imshow(train[0][0].permute(1, 2, 0).numpy())

NameError: name 'Image' is not defined

## Redes convolucionales profundas

1.

In [8]:
import torch.nn as nn
class DWSepConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=True):
        super(DWSepConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=1,
                                   padding=padding, bias=bias, groups = in_channels)
        self.pointwise = nn.Conv2d(in_channels , out_channels, kernel_size=1)
    def forward(self,x):
        D = self.depthwise(x)
        P = self.pointwise(D)
        return P
        

2.

In [9]:
class VGG16DWSep(nn.Module):
    def __init__(self):
        super(VGG16DWSep, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            DWSepConv2d(64,128,kernel_size=3, padding=1),
            nn.ReLU(),
            DWSepConv2d(128,128,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            DWSepConv2d(128,256,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            DWSepConv2d(256,256,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            DWSepConv2d(256,256,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            DWSepConv2d(256,512,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            DWSepConv2d(512,512,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Flatten(),
            nn.Linear(100352, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.7),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512,2))
        
    def forward(self,x):
        return self.layers(x)
         

3.

In [10]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
model = VGG16DWSep()

In [21]:
model.layers[0].weight.data = vgg16.features[0].weight.data
model.layers[0].bias.data = vgg16.features[0].bias.data
model.layers[2].weight.data = vgg16.features[2].weight.data
model.layers[2].bias.data = vgg16.features[2].bias.data

4.

In [44]:
class EarlyStopping():
    def __init__(self, modo='min', paciencia=5, porcentaje=False, tol=0):
        assert(modo == 'min' or modo == 'max')
        
        self.modo = modo
        self.paciencia = paciencia
        self.porcentaje = porcentaje
        self.tol = tol
        self.best_metrica = float('inf') if modo == 'min' else -float('inf')
        self.count = 0
        
    def mejor(self, metrica_validacion):
        
        if(self.porcentaje):
            delta = metrica_validacion/self.best_metrica
            if (self.modo == 'min'):
                return delta < 1 + self.tol 
            else:
                return delta > 1 + self.tol
        else:
            delta = metrica_validacion - self.best_metrica
            if (self.modo == 'min'):
                return delta + self.tol < 0
            else:
                return delta - self.tol > 0
            
    def deberia_parar(self, metrica_validacion):
        if(self.mejor(metrica_validacion)):
            self.count = 0
            self.best_metrica = metrica_validacion
        else:
            self.count+=1
            
        return self.count > self.paciencia

## Interpretabilidad

1.

In [4]:
transform2 = Compose([Resize([229,229]),
                      CenterCrop(299),
                      ToTensor(),
                      Normalize(mean= [0.485, 0.456, 0.406] , std =[0.229, 0.224, 0.225])
                     ])

In [11]:
inceptV3 = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to C:\Users\camil/.cache\torch\checkpoints\inception_v3_google-1a9a5a14.pth


In [16]:
from keras.applications.imagenet_utils import decode_predictions

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [118]:
img=DatasetFolder(root + '/train', loader, extensions, transform2)
img_loader = torch.utils.data.DataLoader(img, batch_size=16, 
                                           sampler=train_sampler)

In [121]:
for i, (X, y) in enumerate(img_loader):
    pred = inceptV3.forward(X)
    break
print(pred)

InceptionOutputs(logits=tensor([[-1.0573,  0.0485,  0.4695,  ..., -0.8977, -0.6715,  5.0489],
        [ 0.6829,  1.9305,  3.8671,  ...,  0.2100, -0.9783, -0.6593],
        [ 0.6373, -0.0721,  0.4821,  ...,  1.1834,  0.4038, -1.7590],
        ...,
        [ 0.3451, -0.5720, -0.2185,  ..., -1.7689, -1.7388, -0.2258],
        [-0.4153, -0.1476,  0.0438,  ...,  0.1550, -0.1917,  0.3499],
        [-0.4170, -0.7890, -0.2276,  ...,  0.4740,  0.3862,  0.5034]],
       grad_fn=<AddmmBackward>), aux_logits=tensor([[-1.1676, -0.9460, -0.4012,  ..., -2.1062, -1.4417,  2.7906],
        [ 0.4403, -0.1480,  2.2743,  ...,  0.4220,  0.0432, -0.7512],
        [ 0.5675,  0.6301, -0.0646,  ...,  0.2240,  1.4711, -1.9378],
        ...,
        [ 0.3880, -0.6275, -0.5096,  ..., -1.1889, -0.7677,  0.1476],
        [-0.8843, -1.4585, -1.3249,  ..., -0.4976,  0.2012,  0.6144],
        [ 0.5492, -2.9335, -0.4468,  ...,  0.3709, -0.6804,  0.5107]],
       grad_fn=<AddmmBackward>))


In [132]:
pred = np.asarray(pred)

In [143]:
pred = np.reshape(pred, (1,2))
pred.shape

(1, 2)

In [146]:
pred.shape

(1, 2)

In [147]:
from keras.applications.imagenet_utils import decode_predictions
decode_predictions(pred)

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 2)